# Distances

In [1]:
from ipynb.fs.full.koselleck import *

[Koselleck] (11:05:19) Alles bereit (+0.0s)


## Words

In [2]:
def cdist_word(word,period_or_periods=None,run_or_runs=None,neighbors=None,num_runs=10,num_proc=1,progress=True):
    try:
        periods,runs=get_periods_runs(period_or_periods,run_or_runs,num_runs=num_runs)

        # load data
        dfdist=get_veclib_word_data(word,progress=progress)
        if neighbors: dfdist=dfdist[[c for c in dfdist.columns if c in set(neighbors)]]
        if not len(dfdist) or not len(dfdist.columns): return pd.DataFrame()
        if progress: print(f'Finished running cdist_word({word})')
        return dfdist
    except ValueError as e:
        print('!!',e)
        return pd.DataFrame()

In [3]:
cdist_word('virtue',num_proc=4,neighbors=['value','virtues'])

[Koselleck] (11:05:20) Loading cdist data for "virtue" (+1.4s)
[Koselleck] (11:05:20) !! Error -3 while decompressing data: invalid code lengths set (+0.0s)
[Koselleck] (11:05:25) Saving dfdist to "/home/ryan/github/koselleck/db/cdists/data.cdists.virtue.pkl.gz" (+4.3s)
[Koselleck] (11:05:34) Removing old data from "/home/ryan/github/koselleck/db/wvecs/db.koselleck.virtue.sqlite" (+9.4s)
[Koselleck] (11:05:34) Finished loading cdist data from sqlite for "virtue" (+0.0s)
[Koselleck] (11:05:35) Finished running cdist_word(virtue) (+1.4s)


value   virtues
word   period    run                    
virtue 1780-1785 1    2.005081  0.731409
       1720-1725 1    1.765799  1.201660
                 2    1.977594  1.314008
                 3    1.747765  1.381870
                 4    1.878719  1.454959
...                        ...       ...
       1895-1900 6    1.328014  0.749701
                 7    1.404102  0.631176
                 8    1.361410  0.829004
                 9    1.284747  0.849949
                 10   1.452586  0.889271

[360 rows x 2 columns]

In [7]:
cdist_word('werwerwerer')

[Koselleck] (18:39:31) Loading cdist data for "werwerwerer" (+2.2s)
[Koselleck] (18:39:31) No file found at /home/ryan/github/koselleck/db/wvecs/db.koselleck.werwerwerer.sqlite (+0.0s)


""


In [9]:
def _cdist_(objd):
    try:
        return cdist_word(**objd)
    except ValueError as e:
        print('!!!!',e,'!!!!')
        return pd.DataFrame()
    
def cdist(word_or_words,period_or_periods=None,run_or_runs=None,neighbors=None,num_runs=10,
          num_proc=1,force=False,progress=True,cache_only=False,cache=False,vl=None):
    index_cols=['word','neighbor','period','run']
    words=tokenize_fast(word_or_words) if type(word_or_words)==str else list(word_or_words)
    objs=[
        dict(
            word=word,
            period_or_periods=period_or_periods,
            run_or_runs=run_or_runs,
            neighbors=neighbors,
            num_runs=num_runs,
            num_proc=num_proc if len(word_or_words)==1 else 1,
            progress=progress if len(word_or_words)==1 else False
        )
        for word in words
    ]
    o=pmap(
        _cdist_,
        objs,
        num_proc=num_proc,
        progress=False if len(words)<2 else progress,
        desc='Computing cosine distances for words'
    )
    return pd.concat(o) if len(o) else pd.DataFrame()


In [11]:
cdist('virtue,virtues,fwefwefwewef')

Computing cosine distances for words [x1]: 100%|██████████| 3/3 [00:00<00:00,  3.47it/s]


a'       a's        aa       aaa       aad  \
word    period    run                                                     
virtue  1780-1785 1    2.162901  2.326049  2.425738  2.198551  2.091443   
        1720-1725 1    2.262305  2.393198  2.337339  2.217301  1.925527   
                  2    2.280775  2.379018  2.454623  2.301432  2.291592   
                  3    2.349260  2.257687  2.240709  2.109919  2.185930   
                  4    2.440110  2.178210  2.304128  2.174020  2.072909   
...                         ...       ...       ...       ...       ...   
virtues 1955-1960 6    2.284276       NaN  1.886938       NaN       NaN   
                  7    1.985536       NaN  1.576267       NaN       NaN   
                  8    2.068463       NaN  1.713967       NaN       NaN   
                  9    2.172709       NaN  1.734996       NaN       NaN   
                  10   2.184628       NaN  1.910268       NaN       NaN   

                            aal       aam     aaron  aaronson       aas  ...  \
word    period    run                                                    ...   
virtue  1780-1785 1    2.146191       NaN       NaN       NaN  1.750717  ...   
        1720-1725 1    2.149955  2.260944  2.360189       NaN  2.157606  ...   
                  2    2.157009       NaN  2.184957       NaN  1.867736  ...   
                  3    2.247387       NaN       NaN       NaN  1.873399  ...   
                  4    2.069204  2.326053  2.239394       NaN  2.147869  ...   
...                         ...       ...       ...       ...       ...  ...   
virtues 1955-1960 6         NaN       NaN       NaN       NaN       NaN  ...   
                  7         NaN       NaN       NaN       NaN       NaN  ...   
                  8         NaN       NaN       NaN       NaN       NaN  ...   
                  9         NaN       NaN       NaN       NaN       NaN  ...   
                  10        NaN       NaN       NaN       NaN       NaN  ...   

                       zoom  zoroaster  zos        zr        zs        zu  \
word    period    run                                                       
virtue  1780-1785 1     NaN        NaN  NaN  2.131796  2.212931       NaN   
        1720-1725 1     NaN        NaN  NaN  2.480936  2.306062  2.181680   
                  2     NaN        NaN  NaN  2.340634  2.300155  2.254788   
                  3     NaN        NaN  NaN  2.368693  2.388964  2.161367   
                  4     NaN        NaN  NaN  2.420269  2.234603  2.246385   
...                     ...        ...  ...       ...       ...       ...   
virtues 1955-1960 6     NaN        NaN  NaN       NaN       NaN       NaN   
                  7     NaN        NaN  NaN       NaN       NaN       NaN   
                  8     NaN        NaN  NaN       NaN       NaN       NaN   
                  9     NaN        NaN  NaN       NaN       NaN       NaN   
                  10    NaN        NaN  NaN       NaN       NaN       NaN   

                       zulu  zulus    zurich  zz  
word    period    run                             
virtue  1780-1785 1     NaN    NaN       NaN NaN  
        1720-1725 1     NaN    NaN       NaN NaN  
                  2     NaN    NaN  1.992242 NaN  
                  3     NaN    NaN  1.854614 NaN  
                  4     NaN    NaN       NaN NaN  
...                     ...    ...       ...  ..  
virtues 1955-1960 6     NaN    NaN  2.197970 NaN  
                  7     NaN    NaN  2.289087 NaN  
                  8     NaN    NaN  2.059180 NaN  
                  9     NaN    NaN  2.485581 NaN  
                  10    NaN    NaN  2.458443 NaN  

[960 rows x 31059 columns]

In [ ]:

# def do_cdist(objd):
#     res=cdist(**objd)
#     if res is None: res=pd.DataFrame()
#     return res

# def cdist(word,period=None,run=None,prefix='cdist',neighbors=None,max_num=10000,num_runs=10,num_proc=1,force=False,progress=True,cache_only=False,cache=False,vl=None):
#     index_cols=['word','neighbor','period','run']
#     argd=dict(
#         word=word,period=period,run=run,prefix=prefix,
#         neighbors=neighbors,max_num=max_num,num_runs=num_runs,
#         num_proc=num_proc,force=force,progress=progress,
#         cache_only=cache_only,cache=cache,vl=vl
#     )
#     odf=None
    
#     if type(word)!=str:
#         objs=[{**argd, **{'word':w, 'progress':False, 'num_proc':1}} for w in word]
#         odf=pd.concat(pmap(do_cdist, objs, num_proc=num_proc, desc='Measuring cosine distances across words', progress=progress, use_threads=False))
#         return odf if not cache_only else pd.DataFrame()
    
#     with get_veclib(prefix=word,folders=['wvecs'],autocommit=False) as vl:
#         dfdist=pd.DataFrame(dict(vl.items())).T
        
        
        
#         if period is None:
#             objs=[{**argd, **{'period':prd, 'progress':False, 'num_proc':1, 'vl':vl}} for prd in get_default_periods()]
#             odf=pd.concat(pmap(do_cdist, objs, num_proc=num_proc, desc='Measuring cosine distances across periods', progress=progress, use_threads=True))
#             return odf if not cache_only else pd.DataFrame()
#         if run is None:
#             objs=[{**argd, **{'run':run+1, 'progress':False, 'num_proc':1, 'vl':vl}} for run in range(num_runs)]
#             odf=pd.concat(pmap(do_cdist, objs, num_proc=num_proc, desc='Measuring cosine distances across runs', progress=progress, use_threads=True))
#             return odf if not cache_only else pd.DataFrame()

#         # load vecs?
#         qstr=f'{period}_{run:02}'
#         sdist=vl.get(qstr)
#         if sdist is None: return pd.DataFrame()
# #         sdist=sdist[set(sdist.index)-{word}]
# #         sdist=sdist*-1 if sdist.min()<-0.1 else sdist
#         if neighbors: sdist=sdist[(set(neighbors) & set(sdist.index))]
#         wddf=pd.DataFrame()
#         wddf['cdist']=sdist
#         wddf=wddf.rename_axis('neighbor').sort_values('cdist')
#         wddf['word']=word
#         wddf['period']=period
#         wddf['run']=run
#         wddf=wddf.reset_index().set_index(index_cols)
#         return wddf


In [ ]:
# dfvecs=vecs('1780-1785',1)
words=get_valid_words()
# words[:100]

In [ ]:
w=random.choice(words)
w

In [ ]:
# %%timeit
# cdist(w,num_proc=8)

In [ ]:
# %%timeit
# cdist(w,num_proc=1)

In [ ]:
# %%timeit
# cdist(w,num_proc=4)

In [ ]:
swords=random.sample(words[:1000],4)
swords

In [ ]:
# %%timeit
# cdist(swords,num_proc=1)

In [ ]:
# %%timeit
# cdist(swords,num_proc=4)

In [ ]:
# cdist(random.sample(words,100),num_proc=4)

In [ ]:
dfdist=cdist('virtues',num_proc=4)
dfdist

In [ ]:
to_nbr('virtues','1720-1725')

In [ ]:
stop

In [ ]:
# cdist(random.sample(words,100),num_proc=4)

In [ ]:
cdist('virtues')

In [ ]:
# # %%timeit
# for word in tqdm(words[:1000]):
#     cdist(word,'1760-1765',progress=False)

In [ ]:
# %%timeit
# cdist('culture','1760-1765',1)

In [ ]:
# dists=cdist('culture',neighbors=['plant','soil','education'],num_proc=1)
# dists

In [ ]:
# distvecs('1770-1775',1)

## Backup

In [ ]:

# def do_cdist(objd):
#     res=cdist(**objd)
#     if res is None: res=pd.DataFrame()
#     return res

# def cdist(word,period=None,run=None,prefix='cdist',neighbors=None,max_num=10000,num_runs=10,num_proc=1,force=False,progress=True,cache_only=False,cache=False,vl=None):
#     index_cols=['word','neighbor','period','run']
#     argd=dict(
#         word=word,period=period,run=run,prefix=prefix,
#         neighbors=neighbors,max_num=max_num,num_runs=num_runs,
#         num_proc=num_proc,force=force,progress=progress,
#         cache_only=cache_only,cache=cache,vl=vl
#     )
#     odf=None
    
#     if type(word)!=str:
#         objs=[{**argd, **{'word':w, 'progress':False, 'num_proc':1}} for w in word]
#         odf=pd.concat(pmap(do_cdist, objs, num_proc=num_proc, desc='Measuring cosine distances across words', progress=progress, use_threads=False))
#         return odf if not cache_only else pd.DataFrame()
    
#     with get_veclib(prefix=word,folders=['wvecs'],autocommit=False) as vl:
#         if period is None:
#             objs=[{**argd, **{'period':prd, 'progress':False, 'num_proc':1, 'vl':vl}} for prd in get_default_periods()]
#             odf=pd.concat(pmap(do_cdist, objs, num_proc=num_proc, desc='Measuring cosine distances across periods', progress=progress, use_threads=True))
#             return odf if not cache_only else pd.DataFrame()
#         if run is None:
#             objs=[{**argd, **{'run':run+1, 'progress':False, 'num_proc':1, 'vl':vl}} for run in range(num_runs)]
#             odf=pd.concat(pmap(do_cdist, objs, num_proc=num_proc, desc='Measuring cosine distances across runs', progress=progress, use_threads=True))
#             return odf if not cache_only else pd.DataFrame()

#         # load vecs?
#         qstr=f'{period}_{run:02}'
#         sdist=vl.get(qstr)
#         if sdist is None: return pd.DataFrame()
# #         sdist=sdist[set(sdist.index)-{word}]
# #         sdist=sdist*-1 if sdist.min()<-0.1 else sdist
#         if neighbors: sdist=sdist[(set(neighbors) & set(sdist.index))]
#         wddf=pd.DataFrame()
#         wddf['cdist']=sdist
#         wddf=wddf.rename_axis('neighbor').sort_values('cdist')
#         wddf['word']=word
#         wddf['period']=period
#         wddf['run']=run
#         wddf=wddf.reset_index().set_index(index_cols)
#         return wddf
